In [ ]:
import pandas as pd

# **_New York City Inspection_**

In [ ]:
# New York City Inscpection CSV
nyc = "../Resources/DOHMH_New_York_City_Restaurant_Inspection_Results.csv"
nyc_df = pd.read_csv(nyc)
nyc_df.head()

In [ ]:
# Create Copy of original CSV
nycdf = nyc_df.copy()

In [ ]:
# Drop any duplicate rows.
nycdf = nycdf.drop_duplicates()

# Replace any missing zipcode with 0 and Remove decimal place in zip code.
nycdf['ZIPCODE'] = nycdf['ZIPCODE'].fillna(0)
nycdf['ZIPCODE'] = nycdf['ZIPCODE'].astype(int)

# Merge Building with Street in one called Address
nycdf['ADDRESS'] = nycdf['BUILDING'] + f" " + nycdf['STREET']

# Unselect Building and Street columns
nycdf = nycdf[['CAMIS', 'DBA', 'BORO', 'ADDRESS', 'ZIPCODE', 'PHONE', 'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 
               'VIOLATION CODE', 'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'SCORE', 'GRADE', 'GRADE DATE', 'RECORD DATE',
               'INSPECTION TYPE']]

nycdf.head()

In [ ]:
# See if there is any other value besides 08/28/2017 in thhe RECORD DATE
# IF THERE IS JUST ONE VALUE, DELETE THE COLUMN
nycdf['RECORD DATE'].value_counts()

In [ ]:
# Remove the RECORD DATE column
nycdf = nycdf = nycdf[['CAMIS', 'DBA', 'BORO', 'ADDRESS', 'ZIPCODE', 'PHONE', 'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 
               'VIOLATION CODE', 'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'SCORE', 'GRADE', 'GRADE DATE', 'INSPECTION TYPE']]
nycdf.head()

In [ ]:
# Reset Index without adding a column
nycdf = nycdf.reset_index(drop=True, inplace=False)
nycdf

# **_San Francisco Inspection_**

In [ ]:
# San Francisco Inscpection CSV
sf = "../Resources/restaurant-scores-live-standard.csv"
sf_df = pd.read_csv(sf)
sf_df

In [ ]:
# Create Copy of original CSV
sfdf = sf_df.copy()

In [ ]:
# Drop any duplicate rows.
sfdf = sfdf.drop_duplicates()

# Unselect unneeded columns business_latitude
sfdf = sfdf[['business_id', 'business_name', 'business_address', 'business_city','business_state', 'business_postal_code',
            'business_phone_number', 'inspection_id', 'inspection_date', 'inspection_score', 'inspection_type', 'violation_id', 
             'violation_description', 'risk_category']]

# Convert the 'business_phone_number' to str instead of float so the whole number is revealed.
sfdf['business_phone_number'] = sfdf['business_phone_number'].astype(str)

# Rename most of the columns with 'business_' since information given is about the business name.
sfdf = sfdf.rename(columns={'business_address': 'address',
                     'business_city': 'city',
                     'business_state': 'state',
                     'business_postal_code': 'postal_code',
                     'business_phone_number': 'phone_number'
                        })

sfdf.head()

In [ ]:
# Remove the decimal (.0) from phone_number
sfdf['phone_number'] = sfdf['phone_number'].str.strip(".0")

# Converted the datetime from the inspection_date to just the date.
sfdf['inspection_date'] = pd.to_datetime(sfdf['inspection_date'])

# Make violation_id unique by removing the 2 sets of numbers that are based from their inspection id. THis allows us to find
# similar results under the same violation_id.
sfdf['violation_id'] =sfdf['violation_id'].str[-6:] 

sfdf

In [ ]:
# Reset Index without adding a column
sfdf = sfdf.reset_index(drop=True, inplace=False)
sfdf

In [17]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [ ]:
protocol = 'postgresql'
username = 'postgres'
password = 'vineyard'
host = 'localhost'
port = 5432
database_name = 'inspections_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [ ]:
engine.table_names()

In [ ]:
# load NYC inspection data
nycdf.to_sql(name='nyc_data', con=engine, if_exists='append', index=False)

In [ ]:
# load SF inspection data
sfdf.to_sql(name='sf_data', con=engine, if_exists='append', index=False)